In [4]:
import multiprocessing
n_cpu = multiprocessing.cpu_count()
print(n_cpu)

12


In [9]:
import numpy as np
import cv2

a = cv2.imread('raw_image.tif')
b = np.uint8(255)
print(type(a))
print(type(b))

<class 'numpy.ndarray'>
<class 'numpy.uint8'>


In [2]:
import numpy as np
i = range(10)
print(np.size(i))

10


In [ ]:
import numpy as np
i = [[1, 2], [3, 4]]
i[end]

[[3 4]
 [1 2]]
[[2 1]
 [4 3]]
